# Make implied volatility surface

In this notebook we construct implied volatility surfaces from empirical as well as model data.

In [1]:
# Standard library imports
import numpy as np
import pandas as pd
import os
import logging
from os.path import dirname as up

# Important paths
code_dir = up(os.getcwd())
deep_cal_dir = up(up(os.getcwd()))

# Problem-specific libraries
from sklearn.linear_model import LinearRegression
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_lets_be_rational.exceptions import BelowIntrinsicException

# Logging stuff
logger = logging.getLogger("iv_surface_construction")
logger.setLevel(logging.INFO)
fh = logging.FileHandler(code_dir + "/logs/iv_surface.log")    
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

# Plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

def set_style():

    sns.set_context("paper")

    sns.set(font='serif')
    
    sns.set_style("white", {
        "font.family": "serif",
        "font.serif": ["Times", "Palatino", "serif"]
    })

In [2]:
set_style()
%matplotlib inline

## Empirical SPX volatility surface

Read in preprocessed SPX data from disk and then use put-call-parity to get market interest rates. For each given maturity, look at all put and call prices and then run a regression to get a rate. 

In [3]:
df = pd.read_csv(deep_cal_dir + '/data/raw_data/processed_spx_calls_all_liquids.csv')
df2 = pd.read_csv(deep_cal_dir + '/data/raw_data/processed_spx_puts_all_liquids.csv')

In [4]:
# Checking if dfs behave like expected:
assert((df.strike == df2.strike).all())
assert(((df.moneyness - 1/df2.moneyness)<1E-10).all())
assert((df['time to maturity (years)'] == df2['time to maturity (years)']).all())

unique_times = df['time to maturity (years)'].unique()
nb_unique_times = unique_times.shape[0]

# Extracting empirical interest rates from put-call-parity by linear regression
# among all options of a given maturity.
for _ in np.arange(nb_unique_times):
    
    T = unique_times[_]
    
    logger.info('Examining time to maturity: {}.'.format(T))
     
    relevant_call_df = df[df['time to maturity (years)'] == T]
    relevant_put_df = df2[df2['time to maturity (years)'] == T]
    
    logger.debug('Call df: {}. Put df: {}.'.format(relevant_call_df, relevant_put_df))
    
    lr = LinearRegression(fit_intercept=True, n_jobs=-1)
    lr.fit(relevant_call_df.strike.values.reshape(-1,1), relevant_call_df.Mid - relevant_put_df.Mid)
    slope = lr.coef_[0]

    r = -1/T*np.log(-slope)
    
    logger.info('Sklearn slope: {}. Computed interest rate: {}.'.format(slope,r))
    
    df.loc[df['time to maturity (years)'] == T, 'rate'] = r


Compute implied volatility from SPX market prices.

In [5]:
# spot price
S = 2731.25

In [6]:
# Wrapper around implied volatility to catch exceptions and set to 0
def iv(price, spot, strike, maturity, rate):

    try:

        return implied_volatility(price, spot, strike, maturity, rate, 'c')

    except BelowIntrinsicException:

        logger.info('Below Intrinsic Exception with' + 
                    'parameters: {}, {}, {}, {}, {}'.format(price, spot, strike,
                                                            maturity, rate))

        return float('NaN')
    
iv = np.vectorize (iv)

In [7]:
df['iv'] = iv(df['Mid'], S, S/df.moneyness, df['time to maturity (years)'], df.rate)

Prepare plotting data and plot

In [12]:
# Preprocessing
moneyness_min = 0.75
moneyness_max = 1.1
maturity_min = 1/365
maturity_max = 0.2

# Remove all options with little or no liquidity
plot_df = df[(df['moneyness']<moneyness_max) & (df['moneyness']>moneyness_min) & (df['time to maturity (years)']<maturity_max) & (df['time to maturity (years)']>maturity_min)]
plot_df = plot_df[plot_df['Open Int'] != 0]

# Remove all options with no computed IV
plot_df = plot_df[plot_df['iv'].notnull()]

logmoneyness = np.log(plot_df.moneyness.values)
maturities = plot_df['time to maturity (years)'].values
iv = plot_df.iv.values

In [76]:
def plot_iv_surface(logmoney, maturities, iv, azim, elev, name):
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    
    ax.azim = azim
    ax.elev = elev
    
    ax.set_xlabel('$m$')
    ax.set_ylabel('$T$')
    ax.set_zlabel('$\sigma_{iv}(m, T)$')

    ax.yaxis.set_ticks(np.linspace(0,0.2, 4, endpoint=False))
    ax.xaxis.set_ticks(np.linspace(-0.25, 0.1, 8, endpoint=True))
    ax.invert_xaxis()

    ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%0.2f'))
    
    ax.plot_trisurf(logmoneyness, maturities, iv, antialiased=True, cmap = plt.cm.Spectral)

    plt.tight_layout()
    fig.savefig(name)
    
    plt.close()

plot_iv_surface(logmoneyness, maturities, iv, 110, 13, 'test.pdf')


In [77]:
for angle in range(0,360, 10):
    plot_iv_surface(logmoneyness, maturities, iv, angle, 10, 'test_{}.pdf'.format(angle))
    